In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.model_v1 import TDConway
import torch
from lib.testing_utilities import get_num_params, boards, visualize_state
from lib.utilities import config, product
import matplotlib.pyplot as plt
%matplotlib inline
plt.ioff()

In [ ]:
get_num_params(TDConway(config))

In [ ]:
boards = torch.stack(boards)
boards.shape

In [ ]:
td_conway = TDConway(config)

In [ ]:
%%timeit
with torch.no_grad():
  td_conway(boards)

In [ ]:
with torch.no_grad():
  res = td_conway(boards)
res

In [ ]:
visualize_state(boards[16])

In [ ]:
with torch.no_grad():
  res = td_conway(boards, get_all_values = True)
res

# Test Move Generation

In [3]:
from lib.testing_utilities import boards, random_board

from lib.move_selection import get_move_options

from lib.timer import Timer

import torch

import numpy as np

In [4]:
# for i, board in enumerate(boards):
#   timer = Timer()
#   options = get_move_options(board, torch.device('cpu'))
#   print(f'Board {i} - {timer}')

In [5]:
from lib.memory import get_memory_usage, format_bytes, print_memory_usage

In [6]:
print_memory_usage()

import gc

print(f'{gc.collect()} items collected')

print_memory_usage()

171.07MiB
60 items collected
171.09MiB


In [ ]:
bad_boards = []

np.random.seed(42)
timer = Timer()

baseline_memory = get_memory_usage()

for i in range(1, 1001):
  if i % 10 == 0:
    print(f'{i} boards done - {timer} - {format_bytes(memory)}')
    timer = Timer()
    
  board = random_board(0.6)
  
  try:
    get_move_options(board, device = torch.device('cpu'), debug = True)
  except RuntimeError as e:
    bad_boards.append((board, e.args[0]))
  
  memory = get_memory_usage()
  if memory > baseline_memory + 2 * 1000 * 1000 * 1000: #2 GB
    raise RuntimeError('Excessive memory')
    